<h1>In this notebook I will use the AQL functionality and implement a chatbot.</h1>

In [ ]:
import sys
sys.path.append("../")
from broncode.apple_classifyer import AppleClassifyer

from sentence_transformers import SentenceTransformer, util
import torchvision.transforms as T
import torch.nn.functional
from torchvision.io import read_image, ImageReadMode
import torch
import pickle as pkl
import os

In [ ]:
# Saved model path
apple_model_path = "../models/AppleClassifyer_86"

In [ ]:
# Load model using pickle
load = open(apple_model_path, "rb")
apple_classifyer = pkl.load(load)

<h3>I've created a folder with 20 apples in it</h3>

-   14 good apples
-   2 blotch apples
-   1 rot apple
-   3 scab apples

In [ ]:
# Lets create a sample for the AQL inspection
aql_sample20 = []
# Path to the folder I created
aql_path = "../informatie/apple_disease_classification/images/Test/AQL_testbatch/"

for filename in os.listdir(aql_path):
    aql_sample20.append(read_image(aql_path+filename, ImageReadMode.RGB)/255)

print(aql_sample20)

In [ ]:
# Resize the images to the shape needed for the model
resize = T.Resize((64,64))
print(aql_sample20[0].shape)

for i in range(len(aql_sample20)):
    aql_sample20[i] = resize(aql_sample20[i])
    # Use unsqueeze to emulate a batch
    aql_sample20[i] = aql_sample20[i].unsqueeze(0)

print(aql_sample20[0].shape)

In [ ]:
# Use aql_classifyer function to inspect the samples. It returns the AQL, amount of good apples and amount of bad apples. It also prints the amount of good apples out of len(sample).
aql, good_amount, bad_amount = apple_classifyer.aql_classifyer(aql_sample20)

print(f"AQL quality from code letter F: {aql}")

In [ ]:
# Set the label and create variables for the chatbot
if aql == 0.4:
    apple_use = "Approved for: grocery/greengrocer"
elif aql == 6.5:
    apple_use = "Approved for apple sauce"
elif aql == 15:
    apple_use = "Approved for apple syrup"
elif aql == "REJECTED":
    apple_use = aql


total_apples = good_amount + bad_amount
percentage = round((good_amount*100) / total_apples)

print(apple_use)

<h2>Implementing chatbot</h2>

Ask one of these questions after running the cell below:
-   What is the AQL of these apples?
-   How many good apples are in this sample?
-   How many bad apples are in this sample?
-   How many apples are in this sample?
-   How do you classify your apples?

In [ ]:
# Call chatbot
chat_model = SentenceTransformer('all-MiniLM-L12-v2')

# Use input function to encode the questions asked
query_embedding = chat_model.encode(input('What is your question?'))

# Give the possible answers
possible_answers = [f"From the {total_apples} apples in this sample {good_amount} are good, so thats {percentage}%.",
                    'How I classify apples is my little secret. >:)',
                    f'There are {(total_apples-good_amount)} bad apples in this sample.',
                    f"There are {total_apples} apples in this sample.",
                    f"The AQL of these apples is {aql}, this means that the use for these apples are: {apple_use}"]

# Encode possible answers
passage_embedding = chat_model.encode(possible_answers)

# Compare question asked with possible answers to find the best answer
answer = util.dot_score(query_embedding, passage_embedding)
# Index highest number
v, idx = torch.max(answer, 1)
# Use the index to give the correct response
print(possible_answers[idx])